In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import select_model_inputs_outputs
from skl2onnx.helpers.onnx_helper import save_onnx_model
from skl2onnx.helpers.onnx_helper import enumerate_model_node_outputs
from skl2onnx.helpers.onnx_helper import load_onnx_model

from stitchnet.stitchonnx.utils import change_input_dim, get_fragments, Net

import numpy as np
from tqdm import tqdm
from glob import glob
from stitchnet.finetuning.finetune2 import finetune_subset
import os
import torch
import random

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

num_classes = 2
num_epochs = 15
feature_extract = True

batch_size = 32
val_batch_size = 64

num_datas = [32, 64, 128, 256, 384, 512]

PyTorch Version:  1.12.1
Torchvision Version:  0.13.1+cu102
device cuda:0


In [6]:
from stitchnet.stitchonnx.utils import load_dl, load_cats_and_dogs_dset
import torch

modelnames = sorted(glob('_models/*.onnx'))

dataloaders_dicts = {}

os.makedirs('_results/finetune2', exist_ok=True)
for i,modelname in tqdm(enumerate(modelnames)):
    model_onnx1 = load_onnx_model(modelname)
    name = os.path.basename(modelname)
    namewithoutext = os.path.splitext(name)[0]
    logfile = f'_results/finetune2/{namewithoutext}.txt'
    if True or not os.path.exists(logfile):
        with open(logfile, 'w') as f:
            for num_data in tqdm(num_datas):
                if num_data not in dataloaders_dicts:
                    dataset_train = load_cats_and_dogs_dset("train")
                    indices = np.random.choice(range(len(dataset_train)), num_data)
                    subset = torch.utils.data.Subset(dataset_train, indices)
                    dataloaders_dicts[num_data] = dict(
                        train=load_dl(subset, batch_size),
                        val=load_dl(load_cats_and_dogs_dset("test"), val_batch_size)
                    )
                
                val_acc_history, val_loss_history, train_acc_history, train_loss_history = finetune_subset(model_onnx1, dataloaders_dicts[num_data], num_classes, num_epochs, num_data, feature_extract)
                f.write(f'{num_data},{val_acc_history[-1]}\n')  
                f.flush()
    # break
    
    

0it [00:00, ?it/s]
  0%|          | 0/6 [00:00<?, ?it/s]

Params to learn:
	 Gemm_2.weight
	 Gemm_2.bias
Epoch 0/14
----------




  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.07s/it]


train Loss: 0.6065 Acc: 0.5000




  0%|          | 0/80 [00:00<?, ?it/s]

  1%|▏         | 1/80 [00:00<01:08,  1.15it/s]

  6%|▋         | 5/80 [00:00<00:11,  6.42it/s]

  9%|▉         | 7/80 [00:01<00:11,  6.39it/s]

 15%|█▌        | 12/80 [00:01<00:05, 12.48it/s]

 19%|█▉        | 15/80 [00:01<00:05, 11.16it/s]

 24%|██▍       | 19/80 [00:02<00:05, 11.40it/s]

 30%|███       | 24/80 [00:02<00:03, 16.40it/s]

 34%|███▍      | 27/80 [00:02<00:03, 13.32it/s]

 39%|███▉      | 31/80 [00:02<00:03, 13.09it/s]

 45%|████▌     | 36/80 [00:02<00:02, 17.79it/s]

 49%|████▉     | 39/80 [00:03<00:02, 14.51it/s]

 54%|█████▍    | 43/80 [00:03<00:02, 13.60it/s]

 60%|██████    | 48/80 [00:03<00:01, 18.03it/s]

 64%|██████▍   | 51/80 [00:04<00:01, 14.63it/s]

 69%|██████▉   | 55/80 [00:04<00:01, 13.50it/s]

 74%|███████▍  | 59/80 [00:04<00:01, 16.93it/s]

 78%|███████▊  | 62/80 [00:04<00:01, 14.53it/s]

 84%|████████▍ | 67/80 [00:05<00:00, 14.33it/s]

 89%|████████▉ | 71/80 [00:05<00:00, 17.55it/s]

 92%|█████████▎| 74/80 [00:05<

...

val Loss: 0.0430 Acc: 0.9825
Epoch 14/14
----------




  0%|          | 0/16 [00:00<?, ?it/s]

  6%|▋         | 1/16 [00:01<00:15,  1.00s/it]

 19%|█▉        | 3/16 [00:01<00:03,  3.34it/s]

 31%|███▏      | 5/16 [00:01<00:01,  5.78it/s]

 44%|████▍     | 7/16 [00:01<00:01,  8.14it/s]

 56%|█████▋    | 9/16 [00:01<00:00, 10.46it/s]

 69%|██████▉   | 11/16 [00:01<00:00, 12.44it/s]

 81%|████████▏ | 13/16 [00:01<00:00, 14.18it/s]

100%|██████████| 16/16 [00:01<00:00,  8.00it/s]


train Loss: 0.0116 Acc: 1.0000




  0%|          | 0/80 [00:00<?, ?it/s]

  1%|▏         | 1/80 [00:01<01:38,  1.25s/it]

  2%|▎         | 2/80 [00:01<00:45,  1.73it/s]

  5%|▌         | 4/80 [00:01<00:21,  3.60it/s]

  6%|▋         | 5/80 [00:01<00:16,  4.47it/s]

  8%|▊         | 6/80 [00:01<00:13,  5.37it/s]

  9%|▉         | 7/80 [00:01<00:11,  6.23it/s]

 10%|█         | 8/80 [00:01<00:10,  6.96it/s]

 11%|█▏        | 9/80 [00:02<00:09,  7.66it/s]

 12%|█▎        | 10/80 [00:02<00:08,  8.18it/s]

 14%|█▍        | 11/80 [00:02<00:07,  8.66it/s]

 15%|█▌        | 12/80 [00:02<00:07,  9.00it/s]

 16%|█▋        | 13/80 [00:02<00:07,  9.24it/s]

 18%|█▊        | 14/80 [00:02<00:07,  9.41it/s]

 19%|█▉        | 15/80 [00:02<00:06,  9.53it/s]

 20%|██        | 16/80 [00:02<00:06,  9.57it/s]

 21%|██▏       | 17/80 [00:02<00:06,  9.68it/s]

 22%|██▎       | 18/80 [00:02<00:06,  9.72it/s]

 24%|██▍       | 19/80 [00:03<00:06,  9.61it/s]

 25%|██▌       | 20/80 [00:03<00:06,  9.49it/s]

 26%|██▋       | 21/80 [00:03<00:06

val Loss: 0.0429 Acc: 0.9825
Training complete in 2m 52s
Best val Acc: 0.982711


In [8]:
# val_acc_history

In [ ]:
# finetuneing stitchnets 

# import pandas as pd
# RESULT_NAME = "1660417292_result_BS_32_MD_16_T_0_TT_0.5_K_2"
# figFolder = f'_results/{RESULT_NAME}_figures'
# os.makedirs(figFolder, exist_ok=True)
# os.makedirs(f'_results/finetune/stitchnets', exist_ok=True)
# data = pd.read_csv(f'_results/{RESULT_NAME}.txt', names=['score','acc','macs','params','path','fragments'])

In [ ]:
# modelname = os.path.splitext(data.iloc[21].path)[0]+'.onnx'
# model_onnx1 = load_onnx_model(modelname)
# model_ft, numsamples, iter_val_acc_history, val_acc_history, val_loss_history, train_acc_history, train_loss_history = finetune(model_onnx1, num_classes, num_epochs, num_iters, batch_size, val_batch_size, feature_extract)
# name = os.path.basename(modelname)
# namewithoutext = os.path.splitext(name)[0]
# model_ft = model_ft.cpu()

# with open(f'_results/finetune/stitchnets/{namewithoutext}.txt', 'w') as f:
#     for n,acc in zip(numsamples, iter_val_acc_history):
#         f.write(f'{n},{acc}\n')   
# torch.onnx.export(model_ft, torch.ones(1,3,224,224), f'_results/finetune/stitchnets/{namewithoutext}_ft.onnx')